## Read Data

In [212]:
import pandas as pd
import numpy as np

movies = pd.read_csv("../ml-latest-small/Movie_Data.csv")
ratings = pd.read_pickle("../ml-latest-small/rating.p")
genres = pd.read_pickle("../ml-latest-small/genres.p")

genres.sample()

ERROR: Could not find a version that satisfies the requirement sklearn.model_selection (from versions: none)
ERROR: No matching distribution found for sklearn.model_selection


,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
80590,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


### Preprocessing

In [167]:
ratings = ratings.merge(genres, left_on = 'movieId', right_index = True)
ratings = ratings.replace(0, np.nan)

### Train Test Split

In [168]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(ratings, random_state=42, test_size=.1)

### Item Profile

In [169]:
genres.loc[4896]

(no genres listed)    0
Action                0
Adventure             1
Animation             0
Children              1
Comedy                0
Crime                 0
Documentary           0
Drama                 0
Fantasy               1
Film-Noir             0
Horror                0
IMAX                  0
Musical               0
Mystery               0
Romance               0
Sci-Fi                0
Thriller              0
War                   0
Western               0
Name: 4896, dtype: int64

### User Profile

In [170]:
genres_cols = genres.columns

In [171]:
for cols in genres_cols:
    train[cols] = train[cols] * train['rating']

train.sample(5)

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1400,14,266,5.0,835441168,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0
80014,501,62,3.0,844973487,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50558,326,49530,5.0,1322251786,NaN,5.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN
12916,82,6338,3.5,1084466483,NaN,NaN,NaN,NaN,3.5,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13820,89,76143,4.5,1520409341,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN


### 각 유저별로 아이템에 준 평균

In [172]:
User_Profile = train.groupby('userId')[genres_cols].mean()
User_Profile.sample(5)

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,,
445,NaN,3.690476,3.657895,3.500000,3.200,4.166667,3.593750,NaN,3.812500,3.958333,NaN,3.500000,3.3,3.0000,4.428571,3.961538,3.875000,3.791667,3.666667,NaN
311,NaN,2.214286,3.166667,3.000000,3.000,1.333333,2.500000,NaN,2.375000,1.666667,NaN,2.750000,3.5,2.0000,1.500000,1.812500,3.166667,2.666667,1.750000,2.75
476,NaN,3.500000,3.733333,4.000000,4.000,3.384615,3.428571,NaN,3.733333,4.125000,NaN,NaN,4.5,4.6000,4.000000,3.590909,4.000000,3.625000,4.333333,4.00
159,NaN,3.500000,3.617647,3.681818,3.750,3.347826,1.600000,NaN,3.225490,3.850000,NaN,NaN,4.0,3.4000,NaN,3.159574,2.416667,2.857143,3.666667,NaN
594,NaN,4.349315,4.227273,4.600000,4.375,4.153846,3.947368,4.5,3.621795,3.394737,NaN,1.608108,4.5,4.4375,2.142857,4.527397,4.321429,3.366071,3.642857,4.00


### My User Profile

In [173]:
User_Profile.loc[400]

(no genres listed)         NaN
Action                4.527778
Adventure             4.269231
Animation             4.500000
Children              4.500000
Comedy                4.666667
Crime                 4.777778
Documentary                NaN
Drama                 4.590909
Fantasy               4.300000
Film-Noir                  NaN
Horror                4.750000
IMAX                  4.200000
Musical                    NaN
Mystery               4.625000
Romance               4.500000
Sci-Fi                4.423077
Thriller              4.666667
War                   4.500000
Western                    NaN
Name: 400, dtype: float64

### Predict

In [187]:
from tqdm.notebook import tqdm

In [194]:
predict = list()

for idx, row in test.iterrows():
    user = row['userId']
    predict.append((User_Profile.loc[user] * row[genres_cols]).mean())

In [209]:
test['predict'] = predict
test.loc[test['predict'].isnull(), 'predict'] = train['rating'].mean()

test['predict'].isna().sum()

0

### Model Validation

In [210]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error()